In [2]:
from mapbox import Geocoder

geocoder = Geocoder(access_token='pk.eyJ1IjoiaHVnaGZpdHpnZXJhbGQiLCJhIjoiY2xkZGhjaG9wMDNqdTNvdDZ5bG80OXZ3YSJ9.6njmVnJyl0zAtMnM9d8duQ')
response = geocoder.forward('4252 Coolidge Ave, Los Angeles, CA 90066')

<Response [200]>


In [6]:
response.json()['features'][0]

{'id': 'address.7082831410212634',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Coolidge Avenue',
 'place_name': '4252 Coolidge Avenue, Los Angeles, California 90066, United States',
 'center': [-118.4171, 34.00101],
 'geometry': {'type': 'Point', 'coordinates': [-118.4171, 34.00101]},
 'address': '4252',
 'context': [{'id': 'neighborhood.394022124', 'text': 'McLaughlin'},
  {'id': 'postcode.300560108', 'text': '90066'},
  {'id': 'place.192407788', 'wikidata': 'Q65', 'text': 'Los Angeles'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [7]:
address = '6605  GREEN VALLEY CIR  UNIT 113	CULVER CITY, CA 90230-8083'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.5063949685794286',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Green Valley Circle',
 'place_name': '6605 Green Valley Circle, Culver City, California 90230, United States',
 'center': [-118.385796, 33.97883],
 'geometry': {'type': 'Point', 'coordinates': [-118.385796, 33.97883]},
 'address': '6605',
 'context': [{'id': 'neighborhood.229092588', 'text': 'Fox Hills'},
  {'id': 'postcode.300904172', 'text': '90230'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [9]:
import usaddress
usaddress.parse(address)

[('6605', 'AddressNumber'),
 ('GREEN', 'StreetName'),
 ('VALLEY', 'StreetName'),
 ('CIR', 'StreetNamePostType'),
 ('UNIT', 'OccupancyType'),
 ('113', 'OccupancyIdentifier'),
 ('CULVER', 'PlaceName'),
 ('CITY,', 'PlaceName'),
 ('CA', 'StateName'),
 ('90230-8083', 'ZipCode')]

In [10]:
address = '9770 CULVER BLVD	CULVER CITY, CA 90232-2703'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.766468067844202',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Culver Boulevard',
 'place_name': '9770 Culver Boulevard, Culver City, California 90232, United States',
 'center': [-118.39559, 34.02163],
 'geometry': {'type': 'Point', 'coordinates': [-118.39559, 34.02163]},
 'address': '9770',
 'context': [{'id': 'neighborhood.375500012', 'text': 'Lucerne'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [11]:
address = '4396 KEYSTONE AVE	CULVER CITY, CA 90232-3437'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.1144368606704674',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Keystone Avenue',
 'place_name': '4396 Keystone Avenue, Culver City, California 90232, United States',
 'center': [-118.39549, 34.009255],
 'geometry': {'type': 'Point', 'coordinates': [-118.39549, 34.009255]},
 'address': '4396',
 'context': [{'id': 'neighborhood.91180268', 'text': 'Carlson'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}